doc (including colab and Kmeans): https://pdh.cnrs.fr/en/download/   
format: https://pdh.cnrs.fr/en/data_format/

In [1]:
import os
import gc
import ast
import fiona
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from pathlib import Path
from osgeo import gdal, osr
from rasterstats import zonal_stats
from rasterio.warp import calculate_default_transform, reproject, Resampling

BASE_DIR = Path('/Users/wenlanzhang/PycharmProjects/Mapineq/src/data-wrangling')
DATA_DIR = Path('/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford')

In [2]:
df = pd.read_csv(f"/Users/wenlanzhang/Downloads/pdh_data.csv") 
df

/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_54347/1034853366.py:1: DtypeWarning: Columns (3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"/Users/wenlanzhang/Downloads/pdh_data.csv")


,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
0,Known PFAS user,45.864286,5.948318,Fortier Beaulieu,Rumilly,France,Industrial site,NaN,Authorities,OSINT,...,https://www.auvergne-rhone-alpes.ars.sante.fr/...,0,Known PFAS Users,[],NaN,NaN,{},NaN,NaN,NaN
1,Known PFAS user,45.838896,5.960591,Salomon,Rumilly,France,Industrial site,NaN,Authorities,OSINT,...,https://www.auvergne-rhone-alpes.ars.sante.fr/...,0,Known PFAS Users,[],NaN,NaN,{},NaN,NaN,NaN
2,Known PFAS user,46.566759,4.904980,Tefal,Tournus,France,Industrial site,NaN,Authorities,OSINT,...,https://www.georisques.gouv.fr/risques/registr...,0,Known PFAS Users,[],NaN,NaN,{},NaN,NaN,NaN
3,Known PFAS user,47.984030,14.272978,Agru,Waldneukirchen,Austria,Industrial site,Manufacture of rubber and plastic products,Company,OSINT,...,https://www.agru.at/en/,0,Known PFAS Users,[],NaN,NaN,"{""maps_link"": ""https://goo.gl/maps/cSD9QcvjzUh...",NaN,NaN,NaN
4,Known PFAS user,48.032000,14.225225,Agru,Bad Hall,Austria,Industrial site,Manufacture of rubber and plastic products,Company,OSINT,...,https://www.agru.at/en/,0,Known PFAS Users,[],NaN,NaN,"{""maps_link"": ""https://goo.gl/maps/NkuHEHiEQbV...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058644,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,386.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-05-22,2024.0
1058645,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""375-22-4"", ""unit"": ""ng/l"", ""subst...",ng/l,0.0,"{""R\u00e9gion"": ""Pays de la Loire"", ""Vol.Moy.J...",Wastewater,2024-05-22,2024.0
1058646,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""2706-90-3"", ""unit"": ""ng/l"", ""subs...",ng/l,110.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-10-23,2024.0
1058647,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,132.0,"{""Flux massique (g)"": 0.0001, ""R\u00e9gion"": ""...",Wastewater,2024-10-23,2024.0


- columns: ['category', 'lat', 'lon', 'name', 'city', 'country', 'type', 'sector', 'source_type', 'data_collection_method', 'source_text', 'source_url', 'dataset_id', 'dataset_name', 'pfas_values', 'unit', 'pfas_sum', 'details', 'matrix', 'date', 'year']
- category: ['Known PFAS user', 'Presumptive', 'Measurement', 'PFAS production facility']  
- unit: [nan, 'ng/l', 'ng/kg', 'ng/kg dw', 'ng/kg fw']  
- data_collection_method: ['OSINT', 'Press inquiry', 'FOI', 'Public data - Download', nan, 'Public data - Scraping', 'Public data - API']  
- matrix: [nan, 'Surface water', 'Groundwater', 'Wastewater', 'Sediment', 'Biota', 'Sea water', 'Drinking water', 'Soil', 'Rainwater', 'Unknown', 'Suspended matter', 'Leachate', 'Sewerage']

dataset_name = dataset_id = 117


In [5]:
# df['dataset_name'].unique() 
df[df['category']=='Presumptive']['dataset_name'].unique()

array(['EPRTR', 'wwtp', 'Major Airports', 'Military bases', 'Paper Mills',
       'Firefighters operational training centers',
       'Norway_municipal_fire_training_sites', 'AFFF from France',
       'Firefighting training and incidents in Flanders'], dtype=object)

In [6]:
df[df['category']=='Measurement']['dataset_name'].unique()

array(['15 textile facilities emitting PFAS', 'Danube samples',
       'PFAS_in_surface_water_2019-2020',
       'POPMON Trinkwasser Daten 260922', 'Flanders DOV',
       'czechia drinking water', 'denmark_ministry_environment',
       'denmark_ministry_environment_wwtp', 'denmark_Sjælland',
       'denmark_Midtjyllands', 'denmark_Syddanmark',
       'denmark_Nordjylland',
       'Finland PFAS in surface water and fish - surface water',
       'PFAS in fish', 'AFFF impacted sites in Finland', 'france_ades',
       'france_naiades', 'ANSES', 'Rhine Aquifer',
       'DE Schleswig Holstein', 'DE Rheinland Pfalz', 'DE Hessen',
       'DE_HB_PFAS_TFA_WRRL_Wasser', 'North sea research papers',
       'frauenhofer_map', 'DE_NRW_StehendesGewaesser',
       'DE_NRW_Fliessgewaesser', 'DE_MV_Biota',
       'ITALIAN NATIONAL DATASET (2018)', 'IT_MITENI_GW', 'IT_MITENI_SW',
       'IT_ARPA_Veneto_PFAS_in_acque_2024_12_04',
       '2021 - Lombardia - Surface Water - Rivers',
       'Lombardia Surfac

In [7]:
df[df['category']=='PFAS production facility']['dataset_name'].unique()

array(['PFAS Producers'], dtype=object)

# Known PFAS user

In [8]:
Known_user = df[df['category'] == 'Known PFAS user']
Known_user = Known_user.drop(columns=['pfas_values', 'pfas_sum', 'unit',
                                      'matrix', 'date', 'year'])
Known_user

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,source_text,source_url,dataset_id,dataset_name,details
0,Known PFAS user,45.864286,5.948318,Fortier Beaulieu,Rumilly,France,Industrial site,NaN,Authorities,OSINT,ARS Rhône-Alpes,https://www.auvergne-rhone-alpes.ars.sante.fr/...,0,Known PFAS Users,{}
1,Known PFAS user,45.838896,5.960591,Salomon,Rumilly,France,Industrial site,NaN,Authorities,OSINT,ARS Rhône-Alpes,https://www.auvergne-rhone-alpes.ars.sante.fr/...,0,Known PFAS Users,{}
2,Known PFAS user,46.566759,4.904980,Tefal,Tournus,France,Industrial site,NaN,Authorities,OSINT,Géorisques,https://www.georisques.gouv.fr/risques/registr...,0,Known PFAS Users,{}
3,Known PFAS user,47.984030,14.272978,Agru,Waldneukirchen,Austria,Industrial site,Manufacture of rubber and plastic products,Company,OSINT,Agru,https://www.agru.at/en/,0,Known PFAS Users,"{""maps_link"": ""https://goo.gl/maps/cSD9QcvjzUh..."
4,Known PFAS user,48.032000,14.225225,Agru,Bad Hall,Austria,Industrial site,Manufacture of rubber and plastic products,Company,OSINT,Agru,https://www.agru.at/en/,0,Known PFAS Users,"{""maps_link"": ""https://goo.gl/maps/NkuHEHiEQbV..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Known PFAS user,44.777860,10.875888,Valvole,Carpi,Italy,Industrial site,Manufacture of rubber and plastic products,Authorities,FOI,FOI document,NaN,0,Known PFAS Users,"{""maps_link"": ""https://goo.gl/maps/rx1Fr42wH66..."
227,Known PFAS user,52.403487,4.829410,Exolum Amsterdam,Amsterdam,Netherlands,Industrial site,NaN,Authorities,FOI,FOI document,NaN,0,Known PFAS Users,{}
228,Known PFAS user,51.967849,4.034241,Neste,Maasvlakte Rotterdam,Netherlands,Industrial site,NaN,Authorities,FOI,FOI document,NaN,0,Known PFAS Users,{}
229,Known PFAS user,51.955693,4.027291,Unified Fire Brigade (Gezamenlijke Brandweer),Maasvlakte Rotterdam,Netherlands,Industrial site,NaN,Authorities,FOI,FOI document,NaN,0,Known PFAS Users,{}


# Measurement_df

matrix: ['Surface water', 'Groundwater', 'Wastewater', 'Sediment', 'Biota', 'Sea water', 'Drinking water', 'Soil', 'Rainwater', 'Unknown', 'Suspended matter', 'Leachate', 'Sewerage']

In [9]:
Measurement_df = df[df['category'] == 'Measurement']
Measurement_df

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
12628,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12629,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,200.0,{},Surface water,NaN,2017.0
12630,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,42400.0,{},Surface water,NaN,2016.0
12631,Measurement,51.771554,6.605953,Gerhard Van Clewe,Hamminkeln-Dingden,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,50.0,{},Surface water,NaN,2017.0
12632,Measurement,49.590101,7.603395,KOB Karl Otto Braun,Wolfstein,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""307-24-4"", ""unit"": ""ng/l"", ""subst...",ng/l,580.0,{},Surface water,NaN,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058644,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,386.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-05-22,2024.0
1058645,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""375-22-4"", ""unit"": ""ng/l"", ""subst...",ng/l,0.0,"{""R\u00e9gion"": ""Pays de la Loire"", ""Vol.Moy.J...",Wastewater,2024-05-22,2024.0
1058646,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""2706-90-3"", ""unit"": ""ng/l"", ""subs...",ng/l,110.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-10-23,2024.0
1058647,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,132.0,"{""Flux massique (g)"": 0.0001, ""R\u00e9gion"": ""...",Wastewater,2024-10-23,2024.0


In [11]:
test =  Measurement_df.head(20)
test

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
12628,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12629,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,200.0,{},Surface water,NaN,2017.0
12630,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,42400.0,{},Surface water,NaN,2016.0
12631,Measurement,51.771554,6.605953,Gerhard Van Clewe,Hamminkeln-Dingden,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,50.0,{},Surface water,NaN,2017.0
12632,Measurement,49.590101,7.603395,KOB Karl Otto Braun,Wolfstein,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""307-24-4"", ""unit"": ""ng/l"", ""subst...",ng/l,580.0,{},Surface water,NaN,2018.0
12633,Measurement,52.319134,7.228233,Rofa,Schüttorf,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,170.0,{},Surface water,NaN,2017.0
12634,Measurement,45.585137,8.053703,Tollegno 1900,Tollegno,Italy,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12635,Measurement,41.407306,-8.387184,Faria & Coelho,Vila Nova de Famalicão,Poland,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,40.0,{},Surface water,NaN,2018.0
12636,Measurement,57.503802,12.698008,Almedahl-Kinna,Kinna,Sweden,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,30.2,{},Surface water,NaN,2017.0
12637,Measurement,48.029770,17.283800,Samorin Kalinkovo_Samorin_Slovakia,Šamorín,Slovakia,Sampling location,NaN,Scientific article,NaN,...,https://doi.org/10.1016/j.jhazmat.2022.129276,11,Danube samples,"[{""cas_id"": ""307-24-4"", ""unit"": ""ng/l"", ""subst...",ng/l,8.5,"{""site_type"": ""Ground water (\u00b5g/L)""}",Groundwater,2019-07-18,2019.0


In [12]:
test['pfas_values'] = test['pfas_values'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
test

/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_54347/3096482464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pfas_values'] = test['pfas_values'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])


,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
12628,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12629,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,200.0,{},Surface water,NaN,2017.0
12630,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,42400.0,{},Surface water,NaN,2016.0
12631,Measurement,51.771554,6.605953,Gerhard Van Clewe,Hamminkeln-Dingden,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,50.0,{},Surface water,NaN,2017.0
12632,Measurement,49.590101,7.603395,KOB Karl Otto Braun,Wolfstein,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '307-24-4', 'unit': 'ng/l', 'subst...",ng/l,580.0,{},Surface water,NaN,2018.0
12633,Measurement,52.319134,7.228233,Rofa,Schüttorf,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,170.0,{},Surface water,NaN,2017.0
12634,Measurement,45.585137,8.053703,Tollegno 1900,Tollegno,Italy,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12635,Measurement,41.407306,-8.387184,Faria & Coelho,Vila Nova de Famalicão,Poland,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '335-67-1', 'unit': 'ng/l', 'subst...",ng/l,40.0,{},Surface water,NaN,2018.0
12636,Measurement,57.503802,12.698008,Almedahl-Kinna,Kinna,Sweden,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{'cas_id': '1763-23-1', 'unit': 'ng/l', 'subs...",ng/l,30.2,{},Surface water,NaN,2017.0
12637,Measurement,48.029770,17.283800,Samorin Kalinkovo_Samorin_Slovakia,Šamorín,Slovakia,Sampling location,NaN,Scientific article,NaN,...,https://doi.org/10.1016/j.jhazmat.2022.129276,11,Danube samples,"[{'cas_id': '307-24-4', 'unit': 'ng/l', 'subst...",ng/l,8.5,"{""site_type"": ""Ground water (\u00b5g/L)""}",Groundwater,2019-07-18,2019.0


In [15]:
test['pfas_values'][12647]

[{'cas_id': '23069-32-1',
  'unit': 'ng/l',
  'substance': 'Methacrylic acid, 2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-nonadecafluorodecyl ester (8CI)',
  'value': 2.5},
 {'cas_id': '4180-26-1',
  'unit': 'ng/l',
  'substance': 'Acrylic acid, 2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9-hexadecafluorononyl ester (7CI,8CI)',
  'value': 0.9},
 {'cas_id': '38565-54-7',
  'unit': 'ng/l',
  'substance': 'Oxirane, (2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,11-heneicosafluoroundecyl)- (9CI)',
  'value': 0.5},
 {'cas_id': '610778-18-2',
  'unit': 'ng/l',
  'substance': '4-Bromo-2-[4,4,5,5,6,6,7,7,8,9,9,9-dodecafluoro-8-(trifluoromethyl)nonyl]phenol',
  'value': 0.3}]

In [14]:
# Step 2: Explode so each item in the list becomes its own row
df_exploded = test.explode('pfas_values')

# Step 3: Normalize the dictionary into columns
pfas_details = pd.json_normalize(df_exploded['pfas_values'])

# Step 4: Combine with original data
df_final = df_exploded.drop(columns=['pfas_values']).reset_index(drop=True)
pfas_details = pfas_details.reset_index(drop=True)

# Concatenate side by side
df_combined = pd.concat([df_final, pfas_details], axis=1)
df_combined

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,unit,pfas_sum,details,matrix,date,year,cas_id,unit,substance,value
0,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,ng/l,130.0,{},Surface water,NaN,2018.0,335-67-1,ng/l,PFOA,90.0
1,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,ng/l,130.0,{},Surface water,NaN,2018.0,1763-23-1,ng/l,PFOS,40.0
2,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,ng/l,200.0,{},Surface water,NaN,2017.0,335-67-1,ng/l,PFOA,200.0
3,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,ng/l,42400.0,{},Surface water,NaN,2016.0,335-67-1,ng/l,PFOA,41400.0
4,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,ng/l,42400.0,{},Surface water,NaN,2016.0,1763-23-1,ng/l,PFOS,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Measurement,43.872140,25.968290,Giurgiu_Giurgiu_Romania,Giurgiu,Romania,Sampling location,NaN,Scientific article,NaN,...,ng/l,81.4,"{""site_type"": ""Influent waste water (\u00b5g/L)""}",Wastewater,2019-08-26,2019.0,423-88-1,ng/l,"Decanoic acid, nonadecafluoro-, ethenyl ester ...",0.5
96,Measurement,44.743972,19.726941,Sabac_Sabac_Serbia,Šabac,Serbia,Sampling location,NaN,Scientific article,NaN,...,ng/l,4.2,"{""site_type"": ""Influent waste water (\u00b5g/L)""}",Wastewater,2019-08-26,2019.0,23069-32-1,ng/l,"Methacrylic acid, 2,2,3,3,4,4,5,5,6,6,7,7,8,8,...",2.5
97,Measurement,44.743972,19.726941,Sabac_Sabac_Serbia,Šabac,Serbia,Sampling location,NaN,Scientific article,NaN,...,ng/l,4.2,"{""site_type"": ""Influent waste water (\u00b5g/L)""}",Wastewater,2019-08-26,2019.0,4180-26-1,ng/l,"Acrylic acid, 2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9-...",0.9
98,Measurement,44.743972,19.726941,Sabac_Sabac_Serbia,Šabac,Serbia,Sampling location,NaN,Scientific article,NaN,...,ng/l,4.2,"{""site_type"": ""Influent waste water (\u00b5g/L)""}",Wastewater,2019-08-26,2019.0,38565-54-7,ng/l,"Oxirane, (2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,1...",0.5


In [8]:
Measurement_exploded = Measurement_df.explode('pfas_values')
# Measurement_exploded = Measurement_exploded[Measurement_exploded['pfas_values'].notna()]  # drop empty rows if any
Measurement_exploded

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
12628,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,NaN,ng/l,130.0,{},Surface water,NaN,2018.0
12629,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,NaN,ng/l,200.0,{},Surface water,NaN,2017.0
12630,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,NaN,ng/l,42400.0,{},Surface water,NaN,2016.0
12631,Measurement,51.771554,6.605953,Gerhard Van Clewe,Hamminkeln-Dingden,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,NaN,ng/l,50.0,{},Surface water,NaN,2017.0
12632,Measurement,49.590101,7.603395,KOB Karl Otto Braun,Wolfstein,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,NaN,ng/l,580.0,{},Surface water,NaN,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058644,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,NaN,ng/l,386.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-05-22,2024.0
1058645,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,NaN,ng/l,0.0,"{""R\u00e9gion"": ""Pays de la Loire"", ""Vol.Moy.J...",Wastewater,2024-05-22,2024.0
1058646,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,NaN,ng/l,110.0,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-10-23,2024.0
1058647,Measurement,NaN,NaN,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,NaN,ng/l,132.0,"{""Flux massique (g)"": 0.0001, ""R\u00e9gion"": ""...",Wastewater,2024-10-23,2024.0


In [10]:
import ast


In [4]:
Measurement_co = Measurement_df[Measurement_df['lat'].notna() & Measurement_df['lon'].notna()]
Measurement_co

,category,lat,lon,name,city,country,type,sector,source_type,data_collection_method,...,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
12628,Measurement,50.808932,3.352552,Maes,Zwevegem,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,130.0,{},Surface water,NaN,2018.0
12629,Measurement,51.016507,4.088303,Tarkett,Dendermonde,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,200.0,{},Surface water,NaN,2017.0
12630,Measurement,51.042282,3.548967,Ververijen Escotex,Deinze,Belgium,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,42400.0,{},Surface water,NaN,2016.0
12631,Measurement,51.771554,6.605953,Gerhard Van Clewe,Hamminkeln-Dingden,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""335-67-1"", ""unit"": ""ng/l"", ""subst...",ng/l,50.0,{},Surface water,NaN,2017.0
12632,Measurement,49.590101,7.603395,KOB Karl Otto Braun,Wolfstein,Germany,Industrial site,Finishing of textiles,Authorities,FOI,...,https://drive.google.com/drive/folders/1CKxJ5Q...,10,15 textile facilities emitting PFAS,"[{""cas_id"": ""307-24-4"", ""unit"": ""ng/l"", ""subst...",ng/l,580.0,{},Surface water,NaN,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056834,Measurement,51.052985,2.341939,HYDROPALE - Rejet unique procédé REF (Aval),DUNKERQUE,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.hauts-de-france.developpement-dura...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,0.0,"{""url_fiche"": ""https://www.georisques.gouv.fr/...",Surface water,2024-03-01,2024.0
1056835,Measurement,51.052985,2.341939,HYDROPALE - Eaux issus du traitement des hydro...,DUNKERQUE,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.hauts-de-france.developpement-dura...,142,France - ICPE,"[{""cas_id"": ""335-76-2"", ""unit"": ""ng/l"", ""subst...",ng/l,0.0,"{""url_fiche"": ""https://www.georisques.gouv.fr/...",Wastewater,2024-04-08,2024.0
1056836,Measurement,51.052985,2.341939,HYDROPALE - Rejet unique procédé REF (Aval),DUNKERQUE,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.hauts-de-france.developpement-dura...,142,France - ICPE,"[{""cas_id"": ""375-22-4"", ""unit"": ""ng/l"", ""subst...",ng/l,0.0,"{""url_fiche"": ""https://www.georisques.gouv.fr/...",Surface water,2024-04-08,2024.0
1056837,Measurement,51.052985,2.341939,HYDROPALE - Eaux issus du traitement des hydro...,DUNKERQUE,France,Sampling location,NaN,Authorities,Public data - Download,...,https://www.hauts-de-france.developpement-dura...,142,France - ICPE,"[{""cas_id"": ""72629-94-8"", ""unit"": ""ng/l"", ""sub...",ng/l,0.0,"{""url_fiche"": ""https://www.georisques.gouv.fr/...",Wastewater,2024-05-14,2024.0


In [5]:
Measurement_co['pfas_values'][12628]

'[{"cas_id": "335-67-1", "unit": "ng/l", "substance": "PFOA", "value": 90.0}, {"cas_id": "1763-23-1", "unit": "ng/l", "substance": "PFOS", "value": 40.0}]'

In [65]:
# Step 1: Convert 'pfas_values' from string to actual list of dicts
Measurement_co['pfas_values'] = Measurement_co['pfas_values'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Step 2: Explode so each item in the list becomes its own row
Measurement_co_exploded = Measurement_co.explode('pfas_values')

# Step 3: Normalize the dictionary into columns
Measurement_co_pfas_details = pd.json_normalize(Measurement_co_exploded['pfas_values'])

# Step 4: Combine with original data
Measurement_co_final = Measurement_co_exploded.drop(columns=['pfas_values']).reset_index(drop=True)
Measurement_co_pfas_details = Measurement_co_pfas_details.reset_index(drop=True)

# Concatenate side by side
Measurement = pd.concat([Measurement_co_final, Measurement_co_pfas_details], axis=1)

Measurement

ValueError: malformed node or string on line 1: <ast.Name object at 0x1431ce320>

## Measurement_noco

In [56]:
Measurement_noco = Measurement_df[Measurement_df['lat'].isna() | Measurement_df['lon'].isna()]
Measurement_noco = Measurement_noco.drop(columns=['lat', 'lon'])
Measurement_noco

,category,name,city,country,type,sector,source_type,data_collection_method,source_text,source_url,dataset_id,dataset_name,pfas_values,unit,pfas_sum,details,matrix,date,year
15258,Measurement,Brunnen Leibnitz I,Leibnitz,Austria,Sampling location,NaN,Authorities,NaN,AGES,https://www.ages.at/,14,POPMON Trinkwasser Daten 260922,"[{""cas_id"": ""375-73-5"", ""unit"": ""ng/l"", ""subst...",ng/l,18.70,{},Drinking water,NaN,2020.0
15259,Measurement,Brunnen Kaindorf I,Kaindorf,Austria,Sampling location,NaN,Authorities,NaN,AGES,https://www.ages.at/,14,POPMON Trinkwasser Daten 260922,"[{""cas_id"": ""27619-97-2"", ""unit"": ""ng/l"", ""sub...",ng/l,177.35,{},Drinking water,NaN,2020.0
15260,Measurement,Brunnen Kaindorf II,Kaindorf,Austria,Sampling location,NaN,Authorities,NaN,AGES,https://www.ages.at/,14,POPMON Trinkwasser Daten 260922,"[{""cas_id"": ""27619-97-2"", ""unit"": ""ng/l"", ""sub...",ng/l,87.46,{},Drinking water,NaN,2020.0
15261,Measurement,Brunnen Kaindorf III,Kaindorf,Austria,Sampling location,NaN,Authorities,NaN,AGES,https://www.ages.at/,14,POPMON Trinkwasser Daten 260922,"[{""cas_id"": ""355-46-4"", ""unit"": ""ng/l"", ""subst...",ng/l,19.96,{},Drinking water,NaN,2020.0
15262,Measurement,Netzprobe Leibnitz Wasserwerk,Leibnitz,Austria,Sampling location,NaN,Authorities,NaN,AGES,https://www.ages.at/,14,POPMON Trinkwasser Daten 260922,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,30.00,{},Drinking water,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058644,Measurement,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,DREAL Pays de la Loire,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,386.00,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-05-22,2024.0
1058645,Measurement,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,DREAL Pays de la Loire,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""375-22-4"", ""unit"": ""ng/l"", ""subst...",ng/l,0.00,"{""R\u00e9gion"": ""Pays de la Loire"", ""Vol.Moy.J...",Wastewater,2024-05-22,2024.0
1058646,Measurement,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,DREAL Pays de la Loire,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""2706-90-3"", ""unit"": ""ng/l"", ""subs...",ng/l,110.00,"{""Flux massique (g)"": 0.0, ""R\u00e9gion"": ""Pay...",Wastewater,2024-10-23,2024.0
1058647,Measurement,AFM RECYCLAGE - Rejets EP (Aval),VERTOU,France,Sampling location,NaN,Authorities,Public data - Download,DREAL Pays de la Loire,https://www.pays-de-la-loire.developpement-dur...,142,France - ICPE,"[{""cas_id"": ""1763-23-1"", ""unit"": ""ng/l"", ""subs...",ng/l,132.00,"{""Flux massique (g)"": 0.0001, ""R\u00e9gion"": ""...",Wastewater,2024-10-23,2024.0
